In [1]:
import warnings 
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
warnings.filterwarnings("ignore")

In [2]:
from scripts.data_read import read_data

In [3]:
df = read_data("../data/cleaned_data.xlsx")

In [ ]:
df_uncleaned = read_data('../data/Week1_challenge_data_source.xlsx')

In [ ]:
df =  df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.head()

In [ ]:
for i,j in enumerate(df_uncleaned.columns):
    print(i,j)

In [ ]:
#add tcp rows to the cleaned dataset 
df['TCP DL Retrans. Vol (Bytes)'] = df_uncleaned['TCP DL Retrans. Vol (Bytes)']
df['TCP UL Retrans. Vol (Bytes)'] = df_uncleaned['TCP UL Retrans. Vol (Bytes)']

In [ ]:
df

In [ ]:
df.info()

In [ ]:
#filling missing values in added column with mean
for i in df.iloc[:,-2:].columns:
    df[i].fillna(df[i].mean(),inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
#create columns
df['Average TCP retransmission'] = df['TCP DL Retrans. Vol (Bytes)']/1000 + df['TCP UL Retrans. Vol (Bytes)']/1000
df['Average RTT'] = df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)']
df['Average throughput'] = df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']

In [ ]:
for i,j in enumerate(df.columns):
    print(i,j)

In [ ]:
#slicing usefull data 
df_experience = df.iloc[:,np.r_[7,26,47:50]]
df_experience

###### aggregation per customer 

In [ ]:
df_aggregated = df_experience.groupby('MSISDN/Number').agg({'Average TCP retransmission':'sum','Average RTT':'sum','Average throughput':'sum'})

#### top 10 

In [ ]:
for i,j in zip(np.arange(0,4).tolist(),df_aggregated.columns):
    print(df_aggregated.iloc[:,i:i+1].sort_values(by=j,ascending=False).head(10))
    print('*'*50)
    

##### bottom 10

In [ ]:
for i,j in zip(np.arange(0,4).tolist(),df_aggregated.columns):
    print(df_aggregated.iloc[:,i:i+1].sort_values(by=j,ascending=False).tail(10))
    print('*'*50)

###### aggregation per  Handset Type

###### top 10 

In [ ]:
df_aggregated_ = df_experience.groupby('Handset Type').agg({'Average throughput':'sum'})

In [ ]:
df_aggregated_.sort_values(by='Average throughput',ascending=False).head(10)

In [ ]:
df_aggregated__ = df_experience.groupby('Handset Type').agg({'Average TCP retransmission':'sum'})

In [ ]:
df_aggregated__.sort_values(by='Average TCP retransmission',ascending=False).head(10)

###### bottom 10

In [ ]:
df_aggregated__.sort_values(by='Average TCP retransmission',ascending=False).tail(10)

In [ ]:
df_aggregated_.sort_values(by='Average throughput',ascending=False).tail(10)

##### clustering  

In [ ]:
df_aggregated

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_aggregated)

In [ ]:
kmeans.labels_

In [ ]:
df_aggregated['clusters'] = kmeans.labels_

In [ ]:
df_aggregated

In [ ]:
df_aggregated['clusters'].value_counts().plot(kind='bar')
plt.xlabel('clusters');

In [ ]:
df_experience.to_csv('../data/experience_data.csv',index=False)